In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data_df = pd.read_excel('drive/MyDrive/FinAI /data/model_train_data_whole_1103.xlsx')

In [ ]:
rating_list = ['AAApi','AAA', 'AAA-','AA+', 'AA','AA-','A+','A','A-','BBB+','BBB','BBB-','BB+','BB','BB-','B+','B','B-','CCC','CC','C']
macro_rating_list = [0,0,0,1,1,1,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3]
rating_list_df = {'ratings': rating_list,'macro_ratings':macro_rating_list}
rating_list_df = pd.DataFrame(rating_list_df)
rating_dict = rating_list_df.reset_index().set_index(['ratings'])['macro_ratings'].to_dict()
data_df['sample ID'] = data_df.index
data_df['macro ratings number'] = data_df['信用评级'].apply(lambda x: rating_dict[x] )

In [ ]:
data_df = data_df.drop(columns = ['Unnamed: 0','full name','short name','wind code','最新报告期','信用评级发布日期', '信用评级', '信用评级与报告差距天数','freeCashFlowOperatingCashFlowRatio','日期差距最小值','ratings number'])

In [ ]:
data_df0 = data_df[data_df['macro ratings number']==0]
data_df1 = data_df[data_df['macro ratings number']==1]
data_df2 = data_df[data_df['macro ratings number']==2]
data_df3 = data_df[data_df['macro ratings number']==3]
data_df0_sample = data_df0.sample(200)
data_df1_sample = data_df1.sample(300)
data_df2_sample = data_df2.sample(200)
data_df3_sample = data_df3
data_df_sample = pd.concat([data_df0_sample,data_df1_sample,data_df2_sample,data_df3_sample])
data_df_sample.sample(frac = 1)

test_set_df = data_df_sample.sample(80)
test_set_df['is test data']=True
test_set_df = test_set_df[['is test data','sample ID']]
data_df = pd.merge(data_df,test_set_df,on = ['sample ID'],how = 'left')
data_df[data_df['is test data']==True]

train_data_df = data_df[data_df['is test data']!=True]
test_data_df = data_df[data_df['is test data']==True]
train_data_df = train_data_df.drop(columns = ['sample ID','is test data'])
test_data_df = test_data_df.drop(columns = ['sample ID','is test data'])

In [ ]:
data_df0 = train_data_df[train_data_df['macro ratings number']==0]
data_df1 = train_data_df[train_data_df['macro ratings number']==1]
data_df2 = train_data_df[train_data_df['macro ratings number']==2]
data_df3 = train_data_df[train_data_df['macro ratings number']==3]



def prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3):

    data_df0_sample = data_df0.sample(200)
    data_df1_sample = data_df1.sample(300)
    data_df2_sample = data_df2.sample(200)
    data_df3_sample = data_df3
    data_df_sample = pd.concat([data_df0_sample,data_df1_sample,data_df2_sample,data_df3_sample])
    data_df_sample.sample(frac = 1)
    col_names = data_df_sample.columns.values.tolist()
    columns = {}
    for idx, n in enumerate(col_names):
        if idx == len(col_names)-1:
            columns[n] = "y"
        else:
            columns[n] = "x%d"%idx
    data_df_sample = data_df_sample.rename(columns=columns)
    x_train = data_df_sample[[i for i in data_df_sample.columns if i!='y']]
    y_train = data_df_sample['y']
    return x_train, y_train




def prepare_testing_samples_df(test_df):
    
    col_names = test_df.columns.values.tolist()
    columns = {}
    for idx, n in enumerate(col_names):
        if idx == len(col_names)-1:
            columns[n] = "y"
        else:
            columns[n] = "x%d"%idx
    test_data_sample_df = test_df.rename(columns=columns)
    x_test = test_data_sample_df[[i for i in test_data_sample_df.columns if i!='y']]
    y_test = test_data_sample_df['y']
    return x_test, y_test



In [ ]:
x_test, y_test = prepare_testing_samples_df(test_data_df)

In [ ]:
(y_test==3).sum()

10

In [ ]:
inputs = keras.Input(shape=(24,), name="digits")
x = layers.Dense(16, activation="sigmoid", name="dense_1")(inputs)
x = layers.Dense(8, activation="sigmoid", name="dense_2")(x)
outputs = layers.Dense(4, activation="softmax", name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
class myCallback(tf.keras.callbacks.Callback): 
  def on_epoch_end(self, epoch, logs={}): 
    if(logs.get('accuracy') > 0.65):
    # print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)
model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
for i in range(0,5):
    x_train, y_train = prepare_training_samples_df(data_df0,data_df1,data_df2,data_df3)
    
    history = model.fit(
        x_train,
        y_train,
        epochs=100,
        # We pass some validation for
        # monitoring validation loss and metrics
        # at the end of each epoch
        callbacks=[callbacks],
        validation_data=(x_test, y_test),
    )
    model.callback

Epoch 1/100
24/25 [===========================>..] - ETA: 0s - loss: 0.8577 - sparse_categorical_accuracy: 0.6016

TypeError: ignored

In [ ]:
np.argmax(model.predict(x_test),axis = 1)

In [ ]:
y_test

In [ ]:
np.argmax(model.predict(x_train),axis = 1)

22/22 [==============================] - 0s 3ms/step


array([3, 0, 1, 3, 0, 0, 2, 1, 1, 3, 2, 2, 0, 1, 2, 2, 1, 0, 0, 0, 1, 1,
       1, 2, 1, 0, 1, 0, 1, 0, 2, 1, 3, 1, 1, 1, 0, 2, 1, 1, 1, 1, 2, 3,
       1, 3, 3, 2, 2, 3, 1, 2, 2, 2, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       3, 2, 1, 0, 2, 0, 0, 2, 1, 0, 0, 2, 1, 1, 0, 0, 2, 2, 2, 1, 0, 2,
       0, 0, 1, 2, 0, 1, 1, 0, 0, 0, 3, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1,
       0, 1, 2, 1, 1, 0, 2, 2, 3, 0, 2, 0, 0, 2, 0, 1, 1, 1, 3, 0, 0, 0,
       0, 2, 0, 1, 2, 2, 0, 0, 2, 0, 2, 2, 1, 1, 0, 2, 1, 1, 0, 3, 1, 2,
       1, 1, 1, 3, 0, 0, 1, 0, 2, 1, 2, 1, 0, 0, 1, 1, 1, 3, 2, 2, 1, 3,
       0, 2, 2, 0, 3, 1, 3, 1, 1, 0, 1, 1, 3, 2, 1, 1, 3, 2, 3, 1, 3, 1,
       2, 1, 1, 0, 0, 1, 3, 2, 1, 3, 0, 1, 0, 1, 3, 1, 1, 1, 2, 1, 2, 2,
       2, 3, 0, 1, 0, 0, 1, 0, 2, 1, 0, 2, 3, 2, 2, 1, 3, 2, 2, 3, 2, 0,
       2, 1, 1, 2, 0, 2, 1, 1, 1, 0, 1, 3, 2, 1, 1, 0, 2, 1, 1, 3, 1, 1,
       0, 0, 0, 0, 3, 1, 2, 0, 3, 1, 1, 1, 3, 0, 2, 0, 1, 1, 0, 0, 1, 1,
       2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 2, 0, 0,

In [ ]:
y_train - np.argmax(model.predict(x_train),axis = 1)

22/22 [==============================] - 0s 2ms/step


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [ ]:
y_test - np.argmax(model.predict(x_test),axis = 1)

3/3 [==============================] - 0s 5ms/step


array([ 0.,  0.,  0., -1.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  1., -1.,
        0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  1.,
        0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0., -2.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  1.,
        0., -1., -1.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
        0.,  2.,  1., -1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        1.])

In [ ]:
data_df.sample(n=5,axis='columns')